In [34]:
from sklearn.neighbors import BallTree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import mean_squared_error
from shapely.geometry import Polygon
from shapely.ops import unary_union
import numpy as np
import pandas as pd
import geopandas as gpd
def get_nearest_neighbors(gdf_obs, gdf_cand = None, k_neighbors=5):

    if gdf_cand is None:
        gdf_cand = gdf_obs 
    src_points = [(x,y) for x,y in zip(gdf_obs.geometry.x , gdf_obs.geometry.y)]
    candidates =  [(x,y) for x,y in zip(gdf_cand.geometry.x , gdf_cand.geometry.y)]

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='euclidean')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors+1)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    closest_gdfs = []
    for k in np.arange(k_neighbors):
        gdf_new = gdf_cand.iloc[indices[k+1]].reset_index()
        del gdf_new['index']
        gdf_new = gdf_new.add_suffix(f'_{k+1}')
        closest_gdfs.append(gdf_new)
        
    closest_gdfs.insert(0,gdf_obs)    
    gdf_final = pd.concat(closest_gdfs,axis=1)

    return gdf_final

def rfsi(gdf, k_neighbors=5, vname = '', ntrees = 150, bound_grid = None, grid_res = 10, seed = None, folds = 5):
    # Definimos el nombre del target y el valor de k para k-nearest neighbors
    target = vname
    
    # Creamos el modelo de Random Forest
    random_forest_model = RandomForestRegressor(n_estimators=ntrees, random_state=seed)
    
    if isinstance(k_neighbors, list):
        print("Buscando el parametro optimo...")
        # Obtenemos los knn vecinos más cercanos
        gdf_nn = get_nearest_neighbors(gdf_obs=gdf,k_neighbors=max(k_neighbors))
        xv = pd.DataFrame(columns=['knn','rmse'])
        for j in k_neighbors:
            knn = j

            # Generamos automáticamente el nombre de las características basadas en el nombre del target y el valor de k
            features = [f'{target}_{i}' for i in range(1, knn + 1)]
            X = gdf_nn[features]
            y = gdf_nn[target]

            # Define los pliegues para la validación cruzada
            kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

            # Realiza la validación cruzada y obtiene las predicciones y los valores reales
            predictions = cross_val_predict(random_forest_model, X, y, cv=kf)

            # Crear un DataFrame con las columnas "Observado" y "Predicho"
            residuals = pd.DataFrame({'observado': y, 'predicho': predictions})

            # Calcular el RMSE utilizando los valores observados y predichos en el DataFrame
            rmse = np.sqrt(mean_squared_error(residuals['observado'], residuals['predicho']))

            #Registrar el valor de rmse
            #xv = pd.concat([xv, pd.DataFrame({'knn': [knn], 'rmse': [rmse]})], ignore_index=True)
            print("Numero vecinos testeado: ", knn, " - rmse resultante: ", round(rmse,3))
            xv.loc[len(xv)] = [knn, rmse]
        
        #Filtrar el menor rmse
        min_rmse = xv.loc[xv['rmse'].idxmin()]

        #Obtener el nuemro de vecinos optimo
        knn = int(min_rmse['knn'])
        rmse = min_rmse['rmse']
        print("Numero vecinos optimo: ", knn, " - rmse: ", round(rmse,3))
    elif isinstance(k_neighbors, int):
        knn = k_neighbors
        # Obtenemos los knn vecinos más cercanos
        gdf_nn = get_nearest_neighbors(gdf_obs=gdf,k_neighbors=knn)
    else:
        return "Tipo de dato no soportado para k_neighbors"
    print("Obteniendo el modelo de prediccion...")
    # Generamos automáticamente el nombre de las características basadas en el nombre del target y el valor de k
    features = [f'{target}_{i}' for i in range(1, knn + 1)]
    X = gdf_nn[features]
    y = gdf_nn[target]

    # Entrena el modelo utilizando todos los datos
    random_forest_model.fit(X, y)
    if isinstance(bound_grid, gpd.GeoDataFrame):
        print("Prediciendo sobre la grilla...")
        # Crea una grilla de celdas con la resolucion especificada en grid_res dentro del área del polígono
        minx, miny, maxx, maxy = bound_grid.total_bounds
        x_coords = range(int(minx), int(maxx), grid_res)
        y_coords = range(int(miny), int(maxy), grid_res)
        grid_cells = []
        for x in x_coords:
            for y in y_coords:
                cell = Polygon([(x, y), (x + grid_res, y), (x + grid_res, y + grid_res), (x, y + grid_res)])
                grid_cells.append(cell)

        grid = gpd.GeoDataFrame(geometry=grid_cells)

        # Reproyecta la geometría de la grilla para que coincida con el CRS del perímetro
        grid = grid.set_crs(bound_grid.crs)

        # Intersecta la grilla con el polígono para eliminar las celdas que están fuera del perímetro
        intersect_grid = gpd.overlay(grid, perimetro, how='intersection')

        # Obtenemos los vecinos mas cercanos de cada celda
        grid = get_nearest_neighbors(gdf_obs=intersect_grid, gdf_cand=gdf, k_neighbors=knn)

        # Aplica el modelo de Random Forest para predecir los valores sobre la grilla
        predict_grid = random_forest_model.predict(grid[features])
        
        # Agrega las predicciones como una nueva columna en el GeoDataFrame de la grilla
        intersect_grid['pred'] = predict_grid

        # Retorna el modelo entrenado y la grilla con las predicciones
        return random_forest_model, intersect_grid
    else:
        # Retorna el modelo entrenado
        return random_forest_model

In [ ]:
obs = gpd.read_file(filename="Data/ECa.gpkg",layer="EC_field_01")
knn = 5
model= rfsi(gdf=obs,k_neighbors=knn,vname='EC90',seed=701408733)

In [17]:
obs = gpd.read_file(filename="Data/ECa.gpkg",layer="EC_field_01")
knn = [15,20,25]
model = rfsi(gdf=obs,k_neighbors=knn,vname='EC30',seed=701408733)


Numero vecinos testeado:  15  - rmse resultante:  1.863
Numero vecinos testeado:  20  - rmse resultante:  1.862
Numero vecinos testeado:  25  - rmse resultante:  1.865
Numero vecinos optimo:  20  - rmse:  1.862


In [35]:
obs = gpd.read_file(filename="Data/ECa.gpkg",layer="EC_field_01")
bound = gpd.read_file(filename="Data/F05_boundary.gpkg")
knn = 20
model,predictions= rfsi(gdf=obs,k_neighbors=knn,vname='EC90',bound_grid=bound,seed=701408733)

Obteniendo el modelo de prediccion...
Prediciendo sobre la grilla...


ValueError: x attribute access only provided for Point geometries